In [3]:
import numpy as np

class SimpleRNN:
    """
    A simple Recurrent Neural Network cell.
    
    The RNN processes sequences one element at a time, maintaining a "hidden state"
    that serves as memory of what it has seen so far.
    
    Think of it like reading a book: as you read each word, you update your
    understanding of the story. The hidden state is your "mental model" of
    the story so far.
    
    Key insight: The SAME weights are used at every time step. This means:
    1. The network can handle sequences of any length
    2. What it learns about processing position 1 applies to position 100
    3. The number of parameters doesn't grow with sequence length
    """
    
    def __init__(self, input_size, hidden_size):
        """
        Initialize the RNN.
        
        Parameters
        ----------
        input_size : int
            Dimension of each input element.
            For text: This would be the word embedding dimension (e.g., 256)
            For time series: This might be 1 (single value) or more (multiple sensors)
        
        hidden_size : int
            Dimension of the hidden state (the "memory").
            Larger = more capacity to remember, but more computation.
            Typical values: 128, 256, 512
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # =====================================================================
        # Weight Initialization
        # =====================================================================
        # We use Xavier initialization (explained in Part 0A: it keeps 
        # gradients stable by scaling weights based on layer sizes)
        
        # Weights for transforming the input
        # Shape: (hidden_size, input_size)
        # These weights determine how the current input affects the hidden state
        scale_xh = np.sqrt(1.0 / input_size)
        self.W_xh = np.random.randn(hidden_size, input_size) * scale_xh
        
        # Weights for transforming the previous hidden state
        # Shape: (hidden_size, hidden_size)  
        # These weights determine how the previous memory affects the new memory
        scale_hh = np.sqrt(1.0 / hidden_size)
        self.W_hh = np.random.randn(hidden_size, hidden_size) * scale_hh
        
        # Bias for the hidden state
        # Shape: (hidden_size, 1)
        self.b_h = np.zeros((hidden_size, 1))
        
        print(f"RNN initialized:")
        print(f"  Input size: {input_size}")
        print(f"  Hidden size: {hidden_size}")
        print(f"  W_xh shape: {self.W_xh.shape} (input → hidden)")
        print(f"  W_hh shape: {self.W_hh.shape} (hidden → hidden)")
        print(f"  Total parameters: {self.W_xh.size + self.W_hh.size + self.b_h.size}")
    
    def step(self, x_t, h_prev, verbose=False):
        """
        Process ONE time step of the sequence.
        
        This is the core RNN computation:
        h_t = tanh(W_xh @ x_t + W_hh @ h_prev + b_h)
        
        Parameters
        ----------
        x_t : numpy array, shape (input_size, 1)
            The input at the current time step.
            For text: The word embedding of the current word.
        
        h_prev : numpy array, shape (hidden_size, 1)
            The hidden state from the previous time step.
            This encodes everything the network "remembers" so far.
        
        verbose : bool
            If True, print intermediate computations.
        
        Returns
        -------
        h_t : numpy array, shape (hidden_size, 1)
            The new hidden state after processing this input.
        """
        
        if verbose:
            print(f"\n  --- RNN Step ---")
            print(f"  Input x_t shape: {x_t.shape}")
            print(f"  Previous hidden h_prev shape: {h_prev.shape}")
        
        # Step 1: Transform the current input
        # This extracts features from the current input
        input_contribution = np.dot(self.W_xh, x_t)
        
        if verbose:
            print(f"  W_xh @ x_t: shape {input_contribution.shape}")
        
        # Step 2: Transform the previous hidden state  
        # This carries forward the memory from previous steps
        memory_contribution = np.dot(self.W_hh, h_prev)
        
        if verbose:
            print(f"  W_hh @ h_prev: shape {memory_contribution.shape}")
        
        # Step 3: Combine and add bias
        # The new hidden state is influenced by BOTH current input AND memory
        combined = input_contribution + memory_contribution + self.b_h
        
        # Step 4: Apply tanh activation
        # tanh squashes values to [-1, 1], which helps with:
        # - Keeping hidden state bounded (doesn't explode)
        # - Introducing non-linearity (can learn complex patterns)
        h_t = np.tanh(combined)
        
        if verbose:
            print(f"  New hidden state h_t: shape {h_t.shape}")
            print(f"  Hidden state range: [{h_t.min():.3f}, {h_t.max():.3f}]")
        
        return h_t
    
    def forward(self, X, verbose=False):
        """
        Process an entire sequence.
        
        Parameters
        ----------
        X : numpy array, shape (sequence_length, input_size)
            The full input sequence.
            Each row is one time step.
        
        verbose : bool
            If True, print step-by-step details.
        
        Returns
        -------
        hidden_states : list of numpy arrays
            The hidden state after each time step.
            hidden_states[-1] is the final "summary" of the entire sequence.
        """
        sequence_length = X.shape[0]
        
        # Initialize hidden state to zeros
        # This represents "no memory yet" at the start
        h = np.zeros((self.hidden_size, 1))
        
        hidden_states = []
        
        if verbose:
            print(f"\nProcessing sequence of length {sequence_length}")
            print("=" * 50)
        
        for t in range(sequence_length):
            # Get input at time t
            # Reshape to (input_size, 1) for matrix multiplication
            x_t = X[t:t+1, :].T
            
            if verbose:
                print(f"\nTime step {t}:")
            
            # Process this time step
            h = self.step(x_t, h, verbose=verbose)
            
            # Store the hidden state
            hidden_states.append(h.copy())
        
        if verbose:
            print("\n" + "=" * 50)
            print(f"Sequence processing complete!")
            print(f"Final hidden state encodes the entire sequence.")
        
        return hidden_states


# =============================================================================
# DEMONSTRATION
# =============================================================================

print("=" * 70)
print("RECURRENT NEURAL NETWORKS (RNNs)")
print("=" * 70)
print("""
THE PROBLEM: Standard neural networks expect FIXED-SIZE inputs.
But sequences (text, audio, time series) have VARIABLE length and ORDER matters.

"dog bites man" ≠ "man bites dog"  (same words, different meaning!)

THE SOLUTION: Process one element at a time, maintaining a "hidden state"
that acts as MEMORY of what we've seen so far.

At each time step t:
  h_t = tanh(W_xh @ x_t + W_hh @ h_prev + b)
  
  - x_t: current input (e.g., word embedding)
  - h_prev: memory from previous steps  
  - h_t: updated memory after seeing this input

KEY INSIGHT: Same weights (W_xh, W_hh) are used at EVERY time step.
This means:
  1. Network can handle ANY sequence length
  2. What it learns at position 1 applies to position 100
  3. Parameters don't grow with sequence length

Let's see it in action:
""")

# Create a simple RNN
rnn = SimpleRNN(input_size=4, hidden_size=3)

# Create a dummy sequence (3 time steps, 4 features each)
# In practice, these would be word embeddings or sensor readings
sequence = np.array([
    [0.1, 0.2, 0.3, 0.4],  # Time step 0
    [0.5, 0.6, 0.7, 0.8],  # Time step 1
    [0.2, 0.1, 0.4, 0.3],  # Time step 2
])

print(f"\nInput sequence shape: {sequence.shape}")
print(f"(3 time steps, 4 features per step)")

# Process the sequence
hidden_states = rnn.forward(sequence, verbose=True)

print(f"\nNumber of hidden states: {len(hidden_states)}")
print(f"Final hidden state (summary of entire sequence):")
print(hidden_states[-1].flatten())

print("""
KEY TAKEAWAY:
The final hidden state h_3 is a fixed-size vector that "summarizes" the 
entire input sequence. This can be used for:
  - Sentiment analysis: sequence → h_final → positive/negative
  - Language modeling: at each step, h_t → predict next word
  - Seq2Seq: encode input → h_final → decode to output

LIMITATION (covered next): Gradients vanish over long sequences.
After ~10-20 steps, early inputs barely affect learning.
This is why LSTM/GRU were invented.
""")

RECURRENT NEURAL NETWORKS (RNNs)

THE PROBLEM: Standard neural networks expect FIXED-SIZE inputs.
But sequences (text, audio, time series) have VARIABLE length and ORDER matters.

"dog bites man" ≠ "man bites dog"  (same words, different meaning!)

THE SOLUTION: Process one element at a time, maintaining a "hidden state"
that acts as MEMORY of what we've seen so far.

At each time step t:
  h_t = tanh(W_xh @ x_t + W_hh @ h_prev + b)

  - x_t: current input (e.g., word embedding)
  - h_prev: memory from previous steps  
  - h_t: updated memory after seeing this input

KEY INSIGHT: Same weights (W_xh, W_hh) are used at EVERY time step.
This means:
  1. Network can handle ANY sequence length
  2. What it learns at position 1 applies to position 100
  3. Parameters don't grow with sequence length

Let's see it in action:

RNN initialized:
  Input size: 4
  Hidden size: 3
  W_xh shape: (3, 4) (input → hidden)
  W_hh shape: (3, 3) (hidden → hidden)
  Total parameters: 24

Input sequence shap

In [5]:
def demonstrate_rnn_vanishing_gradient():
    """
    Show why gradients vanish in RNNs over long sequences.
    """
    print("=" * 70)
    print("THE VANISHING GRADIENT PROBLEM IN RNNs")
    print("=" * 70)
    print("""
WHY THIS MATTERS:

Consider: "The cat, which had been sleeping on the warm windowsill 
all afternoon while the rain poured outside, finally woke up."

The verb "woke" must agree with "cat" (singular) — but they're 18 words apart!
For the RNN to learn this, the gradient from "woke" must flow back to "cat".

THE PROBLEM:

During backpropagation, gradients flow backward through time.
At each step, we multiply by:
  - The derivative of tanh (max 1.0, typically ~0.5)
  - The weight matrix W_hh (typically < 1.0 to avoid explosion)

After N steps: gradient ≈ (0.5 × 0.9)^N = 0.45^N

Let's see what happens:
""")
    
    # tanh derivative: max value is 1.0 (at tanh(0) = 0)
    # For typical values, it's much smaller
    # d/dx tanh(x) = 1 - tanh(x)²
    
    # Simulate gradient flow through time
    # Each time step multiplies gradient by tanh_derivative and W_hh
    
    # Assume typical tanh derivative of ~0.5 and well-initialized W_hh
    tanh_deriv_typical = 0.5
    w_hh_effect = 0.9  # Slightly less than 1
    
    gradient_multiplier_per_step = tanh_deriv_typical * w_hh_effect
    
    print(f"Gradient multiplier per time step: {gradient_multiplier_per_step}")
    print()
    
    gradient = 1.0  # Start with gradient = 1 from the loss
    
    for t in [1, 5, 10, 20, 50, 100]:
        gradient_at_t = gradient_multiplier_per_step ** t
        print(f"After {t:3d} time steps: gradient magnitude = {gradient_at_t:.2e}")
    
    print()
    print("After 100 steps, gradient is essentially ZERO!")
    print("Early words in a long sequence barely get any learning signal.")
    print()
    print("""This is catastrophic for language understanding. In our example sentence,
the RNN can't connect "cat" to "woke" because the gradient vanishes 
over those 18 words.

THE SOLUTION: LSTM (Long Short-Term Memory)

LSTM introduces a "cell state" — a highway for information that uses 
ADDITION instead of multiplication. Gradients can flow through addition
without vanishing. Plus, learnable "gates" control what to remember/forget.

That's what we'll build next.
""")


demonstrate_rnn_vanishing_gradient()

THE VANISHING GRADIENT PROBLEM IN RNNs

WHY THIS MATTERS:

Consider: "The cat, which had been sleeping on the warm windowsill 
all afternoon while the rain poured outside, finally woke up."

The verb "woke" must agree with "cat" (singular) — but they're 18 words apart!
For the RNN to learn this, the gradient from "woke" must flow back to "cat".

THE PROBLEM:

During backpropagation, gradients flow backward through time.
At each step, we multiply by:
  - The derivative of tanh (max 1.0, typically ~0.5)
  - The weight matrix W_hh (typically < 1.0 to avoid explosion)

After N steps: gradient ≈ (0.5 × 0.9)^N = 0.45^N

Let's see what happens:

Gradient multiplier per time step: 0.45

After   1 time steps: gradient magnitude = 4.50e-01
After   5 time steps: gradient magnitude = 1.85e-02
After  10 time steps: gradient magnitude = 3.41e-04
After  20 time steps: gradient magnitude = 1.16e-07
After  50 time steps: gradient magnitude = 4.58e-18
After 100 time steps: gradient magnitude = 2.10e-35


In [6]:
import numpy as np

class LSTM:
    """
    Long Short-Term Memory network.
    
    LSTM solves the vanishing gradient problem of standard RNNs by introducing:
    1. A separate "cell state" that carries information across time steps
    2. Gates that control what information flows in and out
    
    The key insight: Instead of always mixing old and new information,
    let the network LEARN when to remember and when to forget.
    
    Think of it like a notepad:
    - Forget gate: Eraser (which notes to erase?)
    - Input gate: Pen (which new notes to write?)
    - Output gate: Highlighter (which notes are relevant right now?)
    - Cell state: The notepad itself (persistent storage)
    """
    
    def __init__(self, input_size, hidden_size):
        """
        Initialize LSTM.
        
        Parameters
        ----------
        input_size : int
            Dimension of each input element (e.g., word embedding size)
        hidden_size : int
            Dimension of hidden state and cell state
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Combined input: [h_{t-1}, x_t] has size (hidden_size + input_size)
        combined_size = hidden_size + input_size
        
        # Xavier initialization scale
        scale = np.sqrt(1.0 / combined_size)
        
        # =====================================================================
        # Forget Gate: Decides what to erase from cell state
        # =====================================================================
        # Output is between 0 (forget completely) and 1 (keep completely)
        self.W_f = np.random.randn(hidden_size, combined_size) * scale
        self.b_f = np.zeros((hidden_size, 1))
        
        # =====================================================================
        # Input Gate: Decides what new information to store
        # =====================================================================
        self.W_i = np.random.randn(hidden_size, combined_size) * scale
        self.b_i = np.zeros((hidden_size, 1))
        
        # =====================================================================
        # Candidate Values: The new information that COULD be stored
        # =====================================================================
        self.W_c = np.random.randn(hidden_size, combined_size) * scale
        self.b_c = np.zeros((hidden_size, 1))
        
        # =====================================================================
        # Output Gate: Decides what to output from cell state
        # =====================================================================
        self.W_o = np.random.randn(hidden_size, combined_size) * scale
        self.b_o = np.zeros((hidden_size, 1))
        
        total_params = 4 * (self.W_f.size + self.b_f.size)
        print(f"LSTM initialized:")
        print(f"  Input size: {input_size}")
        print(f"  Hidden size: {hidden_size}")
        print(f"  Total parameters: {total_params}")
        print(f"  (Note: 4x more parameters than simple RNN due to 4 gates)")
    
    def sigmoid(self, x):
        """Sigmoid activation: squashes to (0, 1) for gating."""
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def step(self, x_t, h_prev, c_prev, verbose=False):
        """
        Process ONE time step.
        
        Parameters
        ----------
        x_t : numpy array, shape (input_size, 1)
            Current input
        h_prev : numpy array, shape (hidden_size, 1)
            Previous hidden state (what we output last time)
        c_prev : numpy array, shape (hidden_size, 1)
            Previous cell state (our persistent memory)
        verbose : bool
            Print detailed gate activations
        
        Returns
        -------
        h_t : numpy array, shape (hidden_size, 1)
            New hidden state (output)
        c_t : numpy array, shape (hidden_size, 1)
            New cell state (updated memory)
        """
        
        # Concatenate previous hidden state and current input
        # This combined vector is what all gates look at
        combined = np.vstack([h_prev, x_t])
        
        # =====================================================================
        # Step 1: Forget Gate - What to erase from memory?
        # =====================================================================
        f_t = self.sigmoid(np.dot(self.W_f, combined) + self.b_f)
        # f_t is between 0 and 1 for each cell state dimension
        # 0 = completely forget, 1 = completely keep
        
        if verbose:
            print(f"  Forget gate (avg): {f_t.mean():.3f} (0=forget all, 1=keep all)")
        
        # =====================================================================
        # Step 2: Input Gate - What new info to store?
        # =====================================================================
        i_t = self.sigmoid(np.dot(self.W_i, combined) + self.b_i)
        # i_t is between 0 and 1: how much to add
        
        if verbose:
            print(f"  Input gate (avg): {i_t.mean():.3f} (0=ignore, 1=store)")
        
        # =====================================================================
        # Step 3: Candidate Values - The new info that COULD be stored
        # =====================================================================
        c_tilde = np.tanh(np.dot(self.W_c, combined) + self.b_c)
        # c_tilde is between -1 and 1: the candidate new values
        
        if verbose:
            print(f"  Candidate values range: [{c_tilde.min():.3f}, {c_tilde.max():.3f}]")
        
        # =====================================================================
        # Step 4: Update Cell State - The actual memory update
        # =====================================================================
        # This is the key equation! Mostly addition, not multiplication.
        # c_t = (forget some old) + (add some new)
        c_t = f_t * c_prev + i_t * c_tilde
        
        if verbose:
            print(f"  Cell state updated: range [{c_t.min():.3f}, {c_t.max():.3f}]")
        
        # =====================================================================
        # Step 5: Output Gate - What to output?
        # =====================================================================
        o_t = self.sigmoid(np.dot(self.W_o, combined) + self.b_o)
        
        if verbose:
            print(f"  Output gate (avg): {o_t.mean():.3f} (0=hide, 1=expose)")
        
        # =====================================================================
        # Step 6: Compute Hidden State (Output)
        # =====================================================================
        # The hidden state is a filtered version of the cell state
        h_t = o_t * np.tanh(c_t)
        
        if verbose:
            print(f"  Hidden state range: [{h_t.min():.3f}, {h_t.max():.3f}]")
        
        return h_t, c_t
    
    def forward(self, X, verbose=False):
        """
        Process an entire sequence.
        
        Parameters
        ----------
        X : numpy array, shape (sequence_length, input_size)
            The full input sequence
        
        Returns
        -------
        hidden_states : list
            Hidden state after each time step
        cell_states : list
            Cell state after each time step
        """
        sequence_length = X.shape[0]
        
        # Initialize both hidden state and cell state to zeros
        h = np.zeros((self.hidden_size, 1))
        c = np.zeros((self.hidden_size, 1))
        
        hidden_states = []
        cell_states = []
        
        if verbose:
            print(f"\nProcessing sequence of length {sequence_length}")
            print("=" * 60)
        
        for t in range(sequence_length):
            x_t = X[t:t+1, :].T
            
            if verbose:
                print(f"\nTime step {t}:")
            
            h, c = self.step(x_t, h, c, verbose=verbose)
            
            hidden_states.append(h.copy())
            cell_states.append(c.copy())
        
        return hidden_states, cell_states


# =============================================================================
# DEMONSTRATION: LSTM vs RNN on long sequences
# =============================================================================

print("=" * 70)
print("LSTM DEMONSTRATION")
print("=" * 70)

lstm = LSTM(input_size=4, hidden_size=3)

# Create a longer sequence
sequence = np.random.randn(10, 4) * 0.5  # 10 time steps

print(f"\nInput sequence shape: {sequence.shape}")
print(f"(10 time steps, 4 features per step)")

hidden_states, cell_states = lstm.forward(sequence, verbose=True)

print("\n" + "=" * 70)
print("KEY INSIGHT: The cell state can carry information across many steps")
print("because it uses addition, not multiplication!")
print("=" * 70)

LSTM DEMONSTRATION
LSTM initialized:
  Input size: 4
  Hidden size: 3
  Total parameters: 96
  (Note: 4x more parameters than simple RNN due to 4 gates)

Input sequence shape: (10, 4)
(10 time steps, 4 features per step)

Processing sequence of length 10

Time step 0:
  Forget gate (avg): 0.593 (0=forget all, 1=keep all)
  Input gate (avg): 0.464 (0=ignore, 1=store)
  Candidate values range: [-0.381, 0.621]
  Cell state updated: range [-0.157, 0.389]
  Output gate (avg): 0.447 (0=hide, 1=expose)
  Hidden state range: [-0.057, 0.198]

Time step 1:
  Forget gate (avg): 0.523 (0=forget all, 1=keep all)
  Input gate (avg): 0.517 (0=ignore, 1=store)
  Candidate values range: [-0.453, 0.053]
  Cell state updated: range [-0.071, -0.058]
  Output gate (avg): 0.535 (0=hide, 1=expose)
  Hidden state range: [-0.039, -0.031]

Time step 2:
  Forget gate (avg): 0.536 (0=forget all, 1=keep all)
  Input gate (avg): 0.545 (0=ignore, 1=store)
  Candidate values range: [-0.347, -0.028]
  Cell state updat

In [7]:
import numpy as np

def compute_attention(encoder_states, decoder_state, verbose=False):
    """
    Compute attention over encoder states given current decoder state.
    
    This implements "additive attention" (Bahdanau attention):
    - Score each encoder state based on its relevance to the decoder state
    - Normalize scores with softmax to get attention weights
    - Compute weighted sum of encoder states
    
    Parameters
    ----------
    encoder_states : numpy array, shape (seq_len, hidden_size)
        Hidden states from the encoder, one per input position.
        Each row represents what the encoder "understood" at that position.
    
    decoder_state : numpy array, shape (hidden_size,)
        Current decoder hidden state.
        This represents "what we're trying to generate right now."
    
    verbose : bool
        If True, print step-by-step computation.
    
    Returns
    -------
    context : numpy array, shape (hidden_size,)
        Weighted sum of encoder states (the "attended" representation)
    attention_weights : numpy array, shape (seq_len,)
        How much attention is paid to each encoder position (sums to 1)
    """
    seq_len, hidden_size = encoder_states.shape
    
    if verbose:
        print("Computing Attention")
        print("=" * 60)
        print(f"Encoder states: {seq_len} positions, each of size {hidden_size}")
        print(f"Decoder state: size {hidden_size}")
    
    # =========================================================================
    # Step 1: Compute attention scores
    # =========================================================================
    # For each encoder position, compute: "How relevant is this to the decoder?"
    # 
    # Simple approach: dot product between encoder and decoder states
    # Higher dot product = more similar = more relevant
    
    scores = []
    
    for j in range(seq_len):
        # Dot product measures similarity
        score = np.dot(encoder_states[j], decoder_state)
        scores.append(score)
    
    scores = np.array(scores)
    
    if verbose:
        print(f"\nStep 1: Raw attention scores (dot products)")
        for j, score in enumerate(scores):
            print(f"  Position {j}: score = {score:.4f}")
    
    # =========================================================================
    # Step 2: Normalize with softmax
    # =========================================================================
    # Convert scores to probabilities that sum to 1
    # Higher score → higher probability → more attention
    
    # Softmax with numerical stability (subtract max)
    exp_scores = np.exp(scores - np.max(scores))
    attention_weights = exp_scores / np.sum(exp_scores)
    
    if verbose:
        print(f"\nStep 2: Attention weights (softmax of scores)")
        for j, weight in enumerate(attention_weights):
            bar = "█" * int(weight * 40)  # Visual bar
            print(f"  Position {j}: {weight:.4f} {bar}")
        print(f"  Sum of weights: {np.sum(attention_weights):.4f} (should be 1.0)")
    
    # =========================================================================
    # Step 3: Compute weighted sum (context vector)
    # =========================================================================
    # The context vector is a weighted combination of all encoder states
    # Positions with higher attention contribute more
    
    context = np.zeros(hidden_size)
    
    for j in range(seq_len):
        # Each encoder state contributes proportionally to its attention weight
        context += attention_weights[j] * encoder_states[j]
    
    if verbose:
        print(f"\nStep 3: Context vector (weighted sum)")
        print(f"  context = Σ(attention_weight[j] × encoder_state[j])")
        print(f"  Context shape: {context.shape}")
        print(f"  Context values: {context}")
    
    return context, attention_weights


# =============================================================================
# DEMONSTRATION
# =============================================================================

print("=" * 70)
print("ATTENTION MECHANISM DEMONSTRATION")
print("=" * 70)

np.random.seed(42)

# Simulate encoder states for a 5-word sentence
# In practice, these come from an RNN or transformer encoder
seq_len = 5
hidden_size = 4

# Create encoder states (pretend these encode: "The cat sat on mat")
encoder_states = np.array([
    [0.8, 0.1, 0.1, 0.0],   # "The" - article, low information
    [0.1, 0.9, 0.1, 0.1],   # "cat" - subject, high information
    [0.1, 0.1, 0.8, 0.1],   # "sat" - verb
    [0.3, 0.1, 0.1, 0.1],   # "on" - preposition
    [0.1, 0.1, 0.2, 0.9],   # "mat" - object
])

# Simulate decoder state when generating a word related to "cat"
# (we want the model to pay attention to "cat")
decoder_state = np.array([0.2, 0.85, 0.1, 0.1])  # Similar to "cat" encoding

print("\nScenario: Translating to French, currently generating word related to 'cat'")
print("We expect attention to focus on position 1 ('cat')")

context, attention = compute_attention(encoder_states, decoder_state, verbose=True)

print("\n" + "=" * 70)
print("INTERPRETATION")
print("=" * 70)
print(f"The model paid most attention to position 1 ('{['The', 'cat', 'sat', 'on', 'mat'][np.argmax(attention)]}')")
print(f"This context vector can now be used to help generate the French word")

ATTENTION MECHANISM DEMONSTRATION

Scenario: Translating to French, currently generating word related to 'cat'
We expect attention to focus on position 1 ('cat')
Computing Attention
Encoder states: 5 positions, each of size 4
Decoder state: size 4

Step 1: Raw attention scores (dot products)
  Position 0: score = 0.2550
  Position 1: score = 0.8050
  Position 2: score = 0.1950
  Position 3: score = 0.1650
  Position 4: score = 0.2150

Step 2: Attention weights (softmax of scores)
  Position 0: 0.1802 ███████
  Position 1: 0.3123 ████████████
  Position 2: 0.1697 ██████
  Position 3: 0.1647 ██████
  Position 4: 0.1731 ██████
  Sum of weights: 1.0000 (should be 1.0)

Step 3: Context vector (weighted sum)
  context = Σ(attention_weight[j] × encoder_state[j])
  Context shape: (4,)
  Context values: [0.25906809 0.34985006 0.23609905 0.22047984]

INTERPRETATION
The model paid most attention to position 1 ('cat')
This context vector can now be used to help generate the French word


In [8]:
import numpy as np

def self_attention_step_by_step(X, verbose=True):
    """
    Demonstrate self-attention with detailed explanation.
    
    Self-attention allows each position in a sequence to "look at" all other
    positions and compute a weighted combination based on relevance.
    
    The key innovation: Instead of using the same representation for everything,
    we project into three different spaces:
    - Query (Q): "What am I looking for?"
    - Key (K): "What do I contain?"
    - Value (V): "What should I contribute?"
    
    Parameters
    ----------
    X : numpy array, shape (seq_len, d_model)
        Input sequence. Each row is one position's embedding.
        In transformers, d_model is typically 512 or 768.
    
    verbose : bool
        If True, print step-by-step details.
    
    Returns
    -------
    output : numpy array, shape (seq_len, d_model)
        Output sequence after self-attention.
        Each position now contains information gathered from all positions.
    attention_weights : numpy array, shape (seq_len, seq_len)
        Attention weight matrix showing how much each position attends to others.
    """
    
    seq_len, d_model = X.shape
    
    if verbose:
        print("=" * 70)
        print("SELF-ATTENTION STEP BY STEP")
        print("=" * 70)
        print(f"\nInput: {seq_len} positions, each with {d_model} dimensions")
    
    # =========================================================================
    # Step 1: Create learnable projection matrices
    # =========================================================================
    # In practice, these are learned during training.
    # Here we'll use random matrices for demonstration.
    
    # d_k is the dimension of queries and keys (often d_model or d_model/num_heads)
    d_k = d_model
    d_v = d_model
    
    # W_Q projects input to query space
    # W_K projects input to key space  
    # W_V projects input to value space
    np.random.seed(42)
    W_Q = np.random.randn(d_model, d_k) * 0.1
    W_K = np.random.randn(d_model, d_k) * 0.1
    W_V = np.random.randn(d_model, d_v) * 0.1
    
    if verbose:
        print(f"\nStep 1: Projection matrices")
        print(f"  W_Q shape: {W_Q.shape} (projects to Query space)")
        print(f"  W_K shape: {W_K.shape} (projects to Key space)")
        print(f"  W_V shape: {W_V.shape} (projects to Value space)")
    
    # =========================================================================
    # Step 2: Compute Q, K, V for each position
    # =========================================================================
    # Each position gets its own Query, Key, and Value vector
    
    Q = np.dot(X, W_Q)  # Shape: (seq_len, d_k)
    K = np.dot(X, W_K)  # Shape: (seq_len, d_k)
    V = np.dot(X, W_V)  # Shape: (seq_len, d_v)
    
    if verbose:
        print(f"\nStep 2: Compute Q, K, V for each position")
        print(f"  Q = X @ W_Q, shape: {Q.shape}")
        print(f"  K = X @ W_K, shape: {K.shape}")
        print(f"  V = X @ W_V, shape: {V.shape}")
        print(f"\n  Interpretation:")
        print(f"  - Q[i] = What is position {0} looking for?")
        print(f"  - K[i] = What does position {0} contain?")
        print(f"  - V[i] = What should position {0} contribute?")
    
    # =========================================================================
    # Step 3: Compute attention scores
    # =========================================================================
    # For each pair of positions (i, j):
    # score[i,j] = Q[i] · K[j] = "How relevant is position j to position i?"
    
    # Matrix multiplication: Q @ K^T gives all pairwise dot products at once!
    # Shape: (seq_len, d_k) @ (d_k, seq_len) = (seq_len, seq_len)
    scores = np.dot(Q, K.T)
    
    if verbose:
        print(f"\nStep 3: Compute attention scores (Q @ K^T)")
        print(f"  Scores shape: {scores.shape}")
        print(f"  scores[i,j] = How much should position i attend to position j?")
        print(f"\n  Raw scores matrix:")
        for i in range(seq_len):
            row = " ".join([f"{s:6.2f}" for s in scores[i]])
            print(f"    Position {i}: [{row}]")
    
    # =========================================================================
    # Step 4: Scale by sqrt(d_k)
    # =========================================================================
    # Why scale? Dot products grow with dimension size.
    # Large values → softmax outputs very peaked (all weight on one position)
    # Scaling keeps the variance manageable.
    
    scale = np.sqrt(d_k)
    scaled_scores = scores / scale
    
    if verbose:
        print(f"\nStep 4: Scale by √d_k = √{d_k} = {scale:.2f}")
        print(f"  Why? Large dot products make softmax too 'peaky'")
        print(f"  Scaled scores range: [{scaled_scores.min():.2f}, {scaled_scores.max():.2f}]")
    
    # =========================================================================
    # Step 5: Apply softmax (row-wise)
    # =========================================================================
    # Convert scores to probabilities
    # Each row sums to 1: attention_weights[i] sums to 1
    
    def softmax(x, axis=-1):
        exp_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
        return exp_x / np.sum(exp_x, axis=axis, keepdims=True)
    
    attention_weights = softmax(scaled_scores, axis=1)
    
    if verbose:
        print(f"\nStep 5: Apply softmax (convert to probabilities)")
        print(f"  Each row now sums to 1.0")
        print(f"\n  Attention weight matrix:")
        for i in range(seq_len):
            row = " ".join([f"{w:5.3f}" for w in attention_weights[i]])
            max_j = np.argmax(attention_weights[i])
            print(f"    Position {i}: [{row}] → focuses on position {max_j}")
    
    # =========================================================================
    # Step 6: Compute weighted sum of values
    # =========================================================================
    # For each position i:
    # output[i] = sum over j of (attention_weight[i,j] * V[j])
    #
    # Matrix form: attention_weights @ V
    # Shape: (seq_len, seq_len) @ (seq_len, d_v) = (seq_len, d_v)
    
    output = np.dot(attention_weights, V)
    
    if verbose:
        print(f"\nStep 6: Compute weighted sum of values")
        print(f"  output = attention_weights @ V")
        print(f"  Output shape: {output.shape}")
        print(f"\n  Interpretation:")
        print(f"  output[i] now contains information from ALL positions,")
        print(f"  weighted by how relevant each position is to position i.")
    
    return output, attention_weights


# =============================================================================
# DEMONSTRATION
# =============================================================================

# Create a simple sequence (3 positions, 4 dimensions each)
# Pretend this represents: ["The", "cat", "sat"]
# with each word having a 4-dimensional embedding

X = np.array([
    [1.0, 0.0, 0.0, 0.0],   # "The" - article
    [0.0, 1.0, 0.5, 0.0],   # "cat" - noun, animate
    [0.0, 0.0, 0.0, 1.0],   # "sat" - verb
])

output, attention_weights = self_attention_step_by_step(X, verbose=True)

print("\n" + "=" * 70)
print("KEY INSIGHT")
print("=" * 70)
print("""
After self-attention, each position's representation incorporates
information from ALL other positions, weighted by relevance.

This is different from RNNs:
- RNN: Position 3 only "sees" position 1 through position 2
- Self-attention: Position 3 directly attends to position 1

This direct connection is why transformers handle long-range
dependencies so much better than RNNs!
""")

SELF-ATTENTION STEP BY STEP

Input: 3 positions, each with 4 dimensions

Step 1: Projection matrices
  W_Q shape: (4, 4) (projects to Query space)
  W_K shape: (4, 4) (projects to Key space)
  W_V shape: (4, 4) (projects to Value space)

Step 2: Compute Q, K, V for each position
  Q = X @ W_Q, shape: (3, 4)
  K = X @ W_K, shape: (3, 4)
  V = X @ W_V, shape: (3, 4)

  Interpretation:
  - Q[i] = What is position 0 looking for?
  - K[i] = What does position 0 contain?
  - V[i] = What should position 0 contribute?

Step 3: Compute attention scores (Q @ K^T)
  Scores shape: (3, 3)
  scores[i,j] = How much should position i attend to position j?

  Raw scores matrix:
    Position 0: [ -0.03  -0.02   0.02]
    Position 1: [ -0.01  -0.02   0.00]
    Position 2: [  0.02   0.02   0.00]

Step 4: Scale by √d_k = √4 = 2.00
  Why? Large dot products make softmax too 'peaky'
  Scaled scores range: [-0.02, 0.01]

Step 5: Apply softmax (convert to probabilities)
  Each row now sums to 1.0

  Attention 

In [9]:
def positional_encoding(max_seq_len, d_model):
    """
    Compute sinusoidal positional encoding.
    
    Why sinusoidal?
    1. Deterministic (no learning required)
    2. Can extrapolate to longer sequences than seen in training
    3. Relative positions can be computed via linear transformation
    
    Parameters
    ----------
    max_seq_len : int
        Maximum sequence length to generate encodings for
    d_model : int
        Dimension of the model (embedding dimension)
    
    Returns
    -------
    PE : numpy array, shape (max_seq_len, d_model)
        Positional encodings. Add these to word embeddings.
    """
    
    # Create position indices: [0, 1, 2, ..., max_seq_len-1]
    positions = np.arange(max_seq_len)[:, np.newaxis]  # Shape: (max_seq_len, 1)
    
    # Create dimension indices: [0, 1, 2, ..., d_model-1]
    dims = np.arange(d_model)[np.newaxis, :]  # Shape: (1, d_model)
    
    # Compute the angles
    # The division by 10000^(2i/d_model) creates different frequencies
    # for different dimensions
    angles = positions / np.power(10000, (2 * (dims // 2)) / d_model)
    
    # Apply sin to even indices, cos to odd indices
    PE = np.zeros((max_seq_len, d_model))
    PE[:, 0::2] = np.sin(angles[:, 0::2])  # Even dimensions: sin
    PE[:, 1::2] = np.cos(angles[:, 1::2])  # Odd dimensions: cos
    
    return PE


# Visualize positional encoding
PE = positional_encoding(max_seq_len=50, d_model=64)

print("Positional Encoding Visualization")
print("=" * 60)
print(f"Shape: {PE.shape} (50 positions, 64 dimensions)")
print(f"\nEach position gets a unique pattern of sin/cos values.")
print(f"These are ADDED to word embeddings to inject position info.")
print(f"\nPosition 0, first 8 dims: {PE[0, :8].round(3)}")
print(f"Position 1, first 8 dims: {PE[1, :8].round(3)}")
print(f"Position 2, first 8 dims: {PE[2, :8].round(3)}")

Positional Encoding Visualization
Shape: (50, 64) (50 positions, 64 dimensions)

Each position gets a unique pattern of sin/cos values.
These are ADDED to word embeddings to inject position info.

Position 0, first 8 dims: [0. 1. 0. 1. 0. 1. 0. 1.]
Position 1, first 8 dims: [0.841 0.54  0.682 0.732 0.533 0.846 0.409 0.912]
Position 2, first 8 dims: [ 0.909 -0.416  0.997  0.071  0.902  0.431  0.747  0.665]


In [12]:
print("=" * 70)
print("MULTI-HEAD ATTENTION")
print("=" * 70)
print("""
WHY MULTIPLE HEADS?

Single attention: "What should I focus on?"
Multi-head: "Let me look at this from MULTIPLE perspectives simultaneously."

Each head can learn to focus on different types of relationships:
  - Head 1: Subject-verb agreement ("The cats... ARE")
  - Head 2: Pronoun resolution (what does "it" refer to?)
  - Head 3: Nearby words (local context)
  - Head 4: Semantic similarity (related concepts)

HOW IT WORKS:
1. Split Q, K, V into num_heads smaller pieces
2. Run scaled dot-product attention on each piece independently
3. Concatenate all head outputs
4. (Optional) Apply final linear projection W_O

The key insight: by using different learned projections for each head,
we get multiple "views" of the same data. This is more expressive than
a single attention mechanism could ever be.
""")


def multi_head_attention(Q, K, V, num_heads=8, verbose=False):
    """
    Multi-head attention: Run attention multiple times in parallel.
    
    Why multiple heads?
    - Each head can learn to focus on different types of relationships
    - One head might track syntax, another semantics, another proximity
    - More expressive than single attention
    
    Parameters
    ----------
    Q, K, V : numpy arrays, shape (seq_len, d_model)
        Query, Key, Value matrices
    num_heads : int
        Number of parallel attention heads
    verbose : bool
        Print details
    
    Returns
    -------
    output : numpy array, shape (seq_len, d_model)
        Multi-head attention output
    """
    
    seq_len, d_model = Q.shape
    
    # Each head operates on a slice of the dimensions
    # d_k = d_model / num_heads
    assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
    d_k = d_model // num_heads
    
    if verbose:
        print(f"Multi-Head Attention")
        print(f"  {num_heads} heads, each with dimension {d_k}")
    
    head_outputs = []
    
    for head in range(num_heads):
        # Slice the dimensions for this head
        start = head * d_k
        end = start + d_k
        
        Q_h = Q[:, start:end]
        K_h = K[:, start:end]
        V_h = V[:, start:end]
        
        # Standard scaled dot-product attention for this head
        scores = np.dot(Q_h, K_h.T) / np.sqrt(d_k)
        attention_weights = np.exp(scores - np.max(scores, axis=1, keepdims=True))
        attention_weights = attention_weights / attention_weights.sum(axis=1, keepdims=True)
        head_output = np.dot(attention_weights, V_h)
        
        head_outputs.append(head_output)
        
        if verbose:
            print(f"  Head {head}: attention pattern computed")
    
    # Concatenate all head outputs
    # Shape: (seq_len, num_heads * d_k) = (seq_len, d_model)
    output = np.concatenate(head_outputs, axis=1)
    
    # In practice, there's a final linear projection here
    # output = output @ W_O
    
    if verbose:
        print(f"  Concatenated output shape: {output.shape}")
    
    return output


# =============================================================================
# DEMONSTRATION
# =============================================================================

print("\n--- Demo: 5 tokens, 16 dimensions, 4 heads ---\n")

np.random.seed(42)
Q = np.random.randn(5, 16)  # 5 tokens, 16 dims
K = np.random.randn(5, 16)
V = np.random.randn(5, 16)

output = multi_head_attention(Q, K, V, num_heads=4, verbose=True)

print(f"\nInput shape:  (5, 16) — 5 tokens, 16 dimensions")
print(f"Output shape: {output.shape} — same shape, but now each position")
print(f"              contains information gathered from all positions")
print(f"              through 4 different 'lenses' (heads).")

MULTI-HEAD ATTENTION

WHY MULTIPLE HEADS?

Single attention: "What should I focus on?"
Multi-head: "Let me look at this from MULTIPLE perspectives simultaneously."

Each head can learn to focus on different types of relationships:
  - Head 1: Subject-verb agreement ("The cats... ARE")
  - Head 2: Pronoun resolution (what does "it" refer to?)
  - Head 3: Nearby words (local context)
  - Head 4: Semantic similarity (related concepts)

HOW IT WORKS:
1. Split Q, K, V into num_heads smaller pieces
2. Run scaled dot-product attention on each piece independently
3. Concatenate all head outputs
4. (Optional) Apply final linear projection W_O

The key insight: by using different learned projections for each head,
we get multiple "views" of the same data. This is more expressive than
a single attention mechanism could ever be.


--- Demo: 5 tokens, 16 dimensions, 4 heads ---

Multi-Head Attention
  4 heads, each with dimension 4
  Head 0: attention pattern computed
  Head 1: attention pattern co

In [14]:
print("=" * 70)
print("FEED-FORWARD NETWORK (Position-wise)")
print("=" * 70)
print("""
WHAT IT IS:

After attention gathers information from all positions, we need to 
PROCESS that information. The feed-forward network is a simple 2-layer 
neural network applied to each position INDEPENDENTLY.

    x → Linear(d_model → d_ff) → ReLU → Linear(d_ff → d_model) → output

WHY THE EXPANSION?

The hidden dimension d_ff is typically 4× the model dimension.
  - GPT-3: d_model=12288, d_ff=49152 (4×)
  - BERT:  d_model=768, d_ff=3072 (4×)

This "expand then compress" pattern lets the network:
  1. Project to a higher-dimensional space (more room to transform)
  2. Apply non-linearity (ReLU creates complex features)
  3. Project back to original dimension

Think of it as: attention decides WHAT to look at, feed-forward 
decides WHAT TO DO with that information.

POSITION-WISE means: The same weights are applied to each position,
but each position is processed independently (no mixing between positions).
""")


def feed_forward(x, d_ff=2048, verbose=False):
    """
    Position-wise Feed-Forward Network.
    
    This is applied identically to each position independently.
    It's essentially a 2-layer neural network that adds:
    - Non-linearity (ReLU activation)
    - Additional learnable transformations
    
    The hidden dimension (d_ff) is typically 4× the model dimension.
    This expansion allows the model to learn more complex transformations.
    
    Parameters
    ----------
    x : numpy array, shape (seq_len, d_model)
        Input from attention layer
    d_ff : int
        Hidden dimension (typically 4 × d_model)
    verbose : bool
        Print details
    
    Returns
    -------
    output : numpy array, shape (seq_len, d_model)
        Output after feed-forward processing
    """
    
    seq_len, d_model = x.shape
    
    # Initialize weights (in practice, these are learned)
    np.random.seed(42)
    W1 = np.random.randn(d_model, d_ff) * 0.02
    b1 = np.zeros(d_ff)
    W2 = np.random.randn(d_ff, d_model) * 0.02
    b2 = np.zeros(d_model)
    
    if verbose:
        print(f"Feed-Forward Network")
        print(f"  Input: {d_model} dims → Hidden: {d_ff} dims → Output: {d_model} dims")
    
    # First linear transformation + ReLU
    # Expands to higher dimension
    hidden = np.maximum(0, np.dot(x, W1) + b1)  # ReLU activation
    
    # Second linear transformation
    # Projects back to model dimension
    output = np.dot(hidden, W2) + b2
    
    if verbose:
        print(f"  Expansion ratio: {d_ff / d_model}×")
    
    return output


# =============================================================================
# DEMONSTRATION
# =============================================================================

print("--- Demo: 5 tokens, 64 dimensions, expand to 256 ---\n")

x = np.random.randn(5, 64)  # 5 tokens, 64 dims (output from attention)
output = feed_forward(x, d_ff=256, verbose=True)

print(f"\nInput shape:  {x.shape}")
print(f"Output shape: {output.shape}")
print(f"\nEach position was processed independently through:")
print(f"  64 → 256 (expand) → ReLU → 256 → 64 (compress)")

FEED-FORWARD NETWORK (Position-wise)

WHAT IT IS:

After attention gathers information from all positions, we need to 
PROCESS that information. The feed-forward network is a simple 2-layer 
neural network applied to each position INDEPENDENTLY.

    x → Linear(d_model → d_ff) → ReLU → Linear(d_ff → d_model) → output

WHY THE EXPANSION?

The hidden dimension d_ff is typically 4× the model dimension.
  - GPT-3: d_model=12288, d_ff=49152 (4×)
  - BERT:  d_model=768, d_ff=3072 (4×)

This "expand then compress" pattern lets the network:
  1. Project to a higher-dimensional space (more room to transform)
  2. Apply non-linearity (ReLU creates complex features)
  3. Project back to original dimension

Think of it as: attention decides WHAT to look at, feed-forward 
decides WHAT TO DO with that information.

POSITION-WISE means: The same weights are applied to each position,
but each position is processed independently (no mixing between positions).

--- Demo: 5 tokens, 64 dimensions, expand 

In [18]:
print("=" * 70)
print("LAYER NORMALIZATION & RESIDUAL CONNECTIONS")
print("=" * 70)
print("""
TWO TECHNIQUES THAT MAKE DEEP TRANSFORMERS TRAINABLE:

1. LAYER NORMALIZATION
   
   Problem: Activations can have wildly different scales across layers.
   Solution: Normalize each position to have mean=0, variance=1.
   
   For each position independently:
     normalized = (x - mean) / sqrt(variance + eps)
   
   Why Layer Norm (not Batch Norm)?
   - Batch Norm: normalizes across batch → needs large batches, weird at inference
   - Layer Norm: normalizes across features → works with any batch size
   
2. RESIDUAL CONNECTIONS (Skip Connections)
   
   Problem: In deep networks, gradients vanish through many layers.
   Solution: Add the input directly to the output: output = x + sublayer(x)
   
   Why this works:
   - Gradients flow DIRECTLY through the addition (no vanishing!)
   - If sublayer learns nothing useful, network can just pass input through
   - Same idea that made ResNets trainable (50+ layers)

In transformers, we combine both:
   output = LayerNorm(x + Sublayer(x))
""")


def layer_norm(x, gamma=None, beta=None, eps=1e-6):
    """
    Layer Normalization.
    
    Unlike Batch Normalization (normalizes across batch),
    Layer Norm normalizes across features for each position independently.
    
    This is preferred in transformers because:
    1. Works with variable sequence lengths
    2. Consistent behavior for training and inference
    3. Each position is normalized independently
    
    Parameters
    ----------
    x : numpy array, shape (..., d_model)
        Input tensor (last dimension is features)
    gamma : numpy array, shape (d_model,), optional
        Learned scale parameter
    beta : numpy array, shape (d_model,), optional
        Learned shift parameter
    eps : float
        Small constant for numerical stability
    
    Returns
    -------
    normalized : numpy array, same shape as x
    """
    
    # Compute mean and variance over the last dimension (features)
    mean = x.mean(axis=-1, keepdims=True)
    var = x.var(axis=-1, keepdims=True)
    
    # Normalize: zero mean, unit variance
    normalized = (x - mean) / np.sqrt(var + eps)
    
    # Apply learnable scale and shift (if provided)
    if gamma is not None:
        normalized = normalized * gamma
    if beta is not None:
        normalized = normalized + beta
    
    return normalized


def residual_block(x, sublayer_fn, verbose=False):
    """
    Residual connection with layer normalization.
    
    output = LayerNorm(x + Sublayer(x))
    
    The residual connection (adding x) is crucial:
    - Gradients can flow directly through addition
    - Easier to learn identity function if needed
    - Enables training of very deep networks
    
    Parameters
    ----------
    x : numpy array
        Input
    sublayer_fn : callable
        The sub-layer (attention or feed-forward)
    
    Returns
    -------
    output : numpy array
        Output after residual + layer norm
    """
    
    # Apply sublayer
    sublayer_output = sublayer_fn(x)
    
    # Add residual connection
    residual = x + sublayer_output
    
    # Apply layer normalization
    output = layer_norm(residual)
    
    if verbose:
        print(f"  Residual connection: input + sublayer_output")
        print(f"  Layer normalization: stabilize activations")
    
    return output


# =============================================================================
# DEMONSTRATION
# =============================================================================

print("--- Demo: Layer Normalization ---\n")

# Create input with varying scales
x = np.array([
    [100.0, 200.0, 300.0, 400.0],  # Position 0: large values
    [0.01, 0.02, 0.03, 0.04],       # Position 1: tiny values
])

print(f"Before normalization:")
print(f"  Position 0: mean={x[0].mean():.1f}, std={x[0].std():.1f}")
print(f"  Position 1: mean={x[1].mean():.3f}, std={x[1].std():.3f}")

normalized = layer_norm(x)

print(f"\nAfter normalization:")
print(f"  Position 0: mean={normalized[0].mean():.6f}, std={normalized[0].std():.2f}")
print(f"  Position 1: mean={normalized[1].mean():.6f}, std={normalized[1].std():.2f}")
print(f"\nBoth positions now have mean≈0, std≈1 regardless of original scale!")


print("\n" + "-" * 70)
print("--- Demo: Residual Connection ---\n")

# Simulate a sublayer that makes small changes
def dummy_sublayer(x):
    return x * 0.1  # Small modification

x_input = np.array([[1.0, 2.0, 3.0, 4.0]])

# Without residual: just the sublayer output
without_residual = dummy_sublayer(x_input)

# With residual: input + sublayer output
with_residual = x_input + dummy_sublayer(x_input)

print(f"Input:            {x_input[0]}")
print(f"Sublayer output:  {without_residual[0]}")
print(f"With residual:    {with_residual[0]}")
print(f"""
The residual connection preserves the original signal!
Even if sublayer learns nothing useful (outputs zeros), 
the input passes through unchanged.

This is why we can stack 12, 24, or even 96 transformer layers
without gradients vanishing.
""")

LAYER NORMALIZATION & RESIDUAL CONNECTIONS

TWO TECHNIQUES THAT MAKE DEEP TRANSFORMERS TRAINABLE:

1. LAYER NORMALIZATION

   Problem: Activations can have wildly different scales across layers.
   Solution: Normalize each position to have mean=0, variance=1.

   For each position independently:
     normalized = (x - mean) / sqrt(variance + eps)

   Why Layer Norm (not Batch Norm)?
   - Batch Norm: normalizes across batch → needs large batches, weird at inference
   - Layer Norm: normalizes across features → works with any batch size

2. RESIDUAL CONNECTIONS (Skip Connections)

   Problem: In deep networks, gradients vanish through many layers.
   Solution: Add the input directly to the output: output = x + sublayer(x)

   Why this works:
   - Gradients flow DIRECTLY through the addition (no vanishing!)
   - If sublayer learns nothing useful, network can just pass input through
   - Same idea that made ResNets trainable (50+ layers)

In transformers, we combine both:
   output = Layer

In [19]:
def masked_self_attention(Q, K, V, verbose=False):
    """
    Masked self-attention for autoregressive generation.
    
    When generating text left-to-right, position i can only attend
    to positions 0, 1, ..., i-1 (not future positions).
    
    We implement this with a mask that sets future attention scores to -infinity,
    which becomes 0 after softmax.
    
    Parameters
    ----------
    Q, K, V : numpy arrays, shape (seq_len, d_model)
    
    Returns
    -------
    output : numpy array, shape (seq_len, d_model)
    """
    
    seq_len, d_k = Q.shape
    
    # Compute attention scores
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    
    # Create mask: upper triangular matrix of -infinity
    # Position i can attend to positions 0..i only
    mask = np.triu(np.ones((seq_len, seq_len)) * (-1e9), k=1)
    
    if verbose:
        print("Masked Self-Attention")
        print(f"  Mask (upper triangle = -inf, prevents looking ahead):")
        print(f"  Position 0: can only see position 0")
        print(f"  Position 1: can see positions 0, 1")
        print(f"  Position 2: can see positions 0, 1, 2")
        print(f"  etc.")
    
    # Apply mask
    masked_scores = scores + mask
    
    # Softmax (masked positions become ~0)
    attention_weights = np.exp(masked_scores - np.max(masked_scores, axis=1, keepdims=True))
    attention_weights = attention_weights / attention_weights.sum(axis=1, keepdims=True)
    
    # Weighted sum of values
    output = np.dot(attention_weights, V)
    
    return output, attention_weights


# Demonstrate masking
print("=" * 70)
print("MASKED SELF-ATTENTION DEMONSTRATION")
print("=" * 70)

Q = np.random.randn(4, 8)
K = np.random.randn(4, 8)
V = np.random.randn(4, 8)

output, weights = masked_self_attention(Q, K, V, verbose=True)

print("\nAttention weights (rows should only have non-zero values up to diagonal):")
for i in range(4):
    row = " ".join([f"{w:.3f}" for w in weights[i]])
    print(f"  Position {i}: [{row}]")

print("\nNotice: Position 0 can only attend to itself (100%)")
print("        Position 3 can attend to all positions 0-3")

MASKED SELF-ATTENTION DEMONSTRATION
Masked Self-Attention
  Mask (upper triangle = -inf, prevents looking ahead):
  Position 0: can only see position 0
  Position 1: can see positions 0, 1
  Position 2: can see positions 0, 1, 2
  etc.

Attention weights (rows should only have non-zero values up to diagonal):
  Position 0: [1.000 0.000 0.000 0.000]
  Position 1: [0.852 0.148 0.000 0.000]
  Position 2: [0.201 0.063 0.736 0.000]
  Position 3: [0.515 0.347 0.023 0.115]

Notice: Position 0 can only attend to itself (100%)
        Position 3 can attend to all positions 0-3


In [21]:
print("=" * 70)
print("PUTTING IT ALL TOGETHER: THE TRANSFORMER LAYER")
print("=" * 70)
print("""
This is where everything we've learned comes together.

A single Transformer layer combines:

    ┌─────────────────────────────────────────────────────────────┐
    │  Input (seq_len, d_model)                                   │
    │       ↓                                                     │
    │  ┌─────────────────────────────────────────┐                │
    │  │  Multi-Head Self-Attention              │                │
    │  │  "Look at all positions, focus on       │                │
    │  │   what's relevant from each"            │                │
    │  └─────────────────────────────────────────┘                │
    │       ↓                                                     │
    │  Add & Norm  (residual + layer norm)                        │
    │       ↓                                                     │
    │  ┌─────────────────────────────────────────┐                │
    │  │  Feed-Forward Network                   │                │
    │  │  "Process each position independently"  │                │
    │  └─────────────────────────────────────────┘                │
    │       ↓                                                     │
    │  Add & Norm  (residual + layer norm)                        │
    │       ↓                                                     │
    │  Output (seq_len, d_model) — same shape!                    │
    └─────────────────────────────────────────────────────────────┘

KEY INSIGHTS:
  - Input and output have the SAME shape → can stack many layers
  - Attention: positions communicate with each other
  - Feed-forward: each position processed independently
  - Residual connections: gradients flow freely (no vanishing!)
  - Layer norm: keeps activations stable

Real transformers stack 6-96 of these layers:
  - BERT-base: 12 layers
  - GPT-3: 96 layers
  - Each layer refines the representations further
""")


class TransformerLayer:
    """
    A single Transformer encoder layer.
    
    Components:
    1. Multi-head self-attention (look at all positions)
    2. Residual connection + Layer normalization
    3. Feed-forward network (process each position)
    4. Residual connection + Layer normalization
    
    The full Transformer stacks N of these layers (typically 6-12).
    """
    
    def __init__(self, d_model, num_heads, d_ff):
        """
        Parameters
        ----------
        d_model : int
            Model dimension (embedding size)
        num_heads : int  
            Number of attention heads
        d_ff : int
            Feed-forward hidden dimension
        """
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        
        # Initialize weights (simplified - real implementation uses proper init)
        np.random.seed(42)
        self.W_Q = np.random.randn(d_model, d_model) * 0.02
        self.W_K = np.random.randn(d_model, d_model) * 0.02
        self.W_V = np.random.randn(d_model, d_model) * 0.02
        self.W_O = np.random.randn(d_model, d_model) * 0.02
        
        self.W_ff1 = np.random.randn(d_model, d_ff) * 0.02
        self.W_ff2 = np.random.randn(d_ff, d_model) * 0.02
    
    def self_attention(self, x):
        """Multi-head self-attention."""
        Q = np.dot(x, self.W_Q)
        K = np.dot(x, self.W_K)
        V = np.dot(x, self.W_V)
        
        # Scaled dot-product attention
        d_k = self.d_model // self.num_heads
        scores = np.dot(Q, K.T) / np.sqrt(d_k)
        weights = np.exp(scores - np.max(scores, axis=1, keepdims=True))
        weights = weights / weights.sum(axis=1, keepdims=True)
        
        attended = np.dot(weights, V)
        output = np.dot(attended, self.W_O)
        
        return output
    
    def feed_forward(self, x):
        """Position-wise feed-forward network."""
        hidden = np.maximum(0, np.dot(x, self.W_ff1))  # ReLU
        output = np.dot(hidden, self.W_ff2)
        return output
    
    def forward(self, x, verbose=False):
        """
        Forward pass through one transformer layer.
        
        Parameters
        ----------
        x : numpy array, shape (seq_len, d_model)
            Input embeddings
        
        Returns
        -------
        output : numpy array, shape (seq_len, d_model)
            Processed embeddings
        """
        
        if verbose:
            print("\nTransformer Layer Forward Pass")
            print("=" * 50)
        
        # Sub-layer 1: Self-attention
        if verbose:
            print("1. Multi-head self-attention...")
        attention_output = self.self_attention(x)
        
        # Add & Norm
        x = layer_norm(x + attention_output)
        if verbose:
            print("   + Residual connection + Layer Norm")
        
        # Sub-layer 2: Feed-forward
        if verbose:
            print("2. Feed-forward network...")
        ff_output = self.feed_forward(x)
        
        # Add & Norm
        output = layer_norm(x + ff_output)
        if verbose:
            print("   + Residual connection + Layer Norm")
            print(f"   Output shape: {output.shape}")
        
        return output


# =============================================================================
# DEMONSTRATION: Full Transformer Layer
# =============================================================================

print("=" * 70)
print("COMPLETE TRANSFORMER LAYER DEMONSTRATION")
print("=" * 70)

# Create a transformer layer
layer = TransformerLayer(d_model=64, num_heads=8, d_ff=256)

# Create input sequence (5 tokens, 64 dimensions each)
x = np.random.randn(5, 64)

print(f"\nInput shape: {x.shape} (5 tokens, 64 dimensions)")

# Process through transformer layer
output = layer.forward(x, verbose=True)

print(f"\nOutput shape: {output.shape}")
print("""
WHAT JUST HAPPENED:

1. Self-attention looked at ALL positions and computed relevance weights
2. Each position gathered information from relevant positions  
3. Residual connection preserved the original signal
4. Layer norm stabilized the activations
5. Feed-forward processed each position independently
6. Another residual + layer norm

The output has the SAME SHAPE as input — this is crucial!
It means we can stack as many layers as we want.

Stack 12 of these → BERT
Stack 96 of these → GPT-3

Each additional layer refines the representations,
building increasingly sophisticated understanding of the input.
""")

PUTTING IT ALL TOGETHER: THE TRANSFORMER LAYER

This is where everything we've learned comes together.

A single Transformer layer combines:

    ┌─────────────────────────────────────────────────────────────┐
    │  Input (seq_len, d_model)                                   │
    │       ↓                                                     │
    │  ┌─────────────────────────────────────────┐                │
    │  │  Multi-Head Self-Attention              │                │
    │  │  "Look at all positions, focus on       │                │
    │  │   what's relevant from each"            │                │
    │  └─────────────────────────────────────────┘                │
    │       ↓                                                     │
    │  Add & Norm  (residual + layer norm)                        │
    │       ↓                                                     │
    │  ┌─────────────────────────────────────────┐                │
    │  │  Feed-Forward Network            